# Prject:3 Multi-agent Collaboration for Financial Analysis

 agents collaborate with each other.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [5]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [6]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [7]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [8]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [9]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [10]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [11]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [12]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [13]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [14]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [15]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to delegate the task of continuously monitoring and analyzing market data for AAPL to the Data Analyst, as they have the expertise in statistical modeling and machine learning needed for this task.

Action: Delegate work to co-worker
Action Input: {"coworker": "Data Analyst", "task": "Continuously monitor and analyze market data for AAPL using statistical modeling and machine learning", "context": "This task involves tracking and analyzing market data for the stock AAPL in order to identify trends and predict market movements. Statistical modeling and machine learning techniques should be utilized to achieve this."}


> Entering new CrewAgentExecutor chain...
I need to gather real-time market data for AAPL 

Final Answer: 
Apple Inc. (AAPL) Stock Price, News, Quote & History - Yahoo Finance Oops, something went wrong News Today's news US Politics World Tech Reviews and deals Audio Computing Gaming Health Home Phones Science TVs Climate change Health Science Originals The 360 Newsletters Life Health COVID-19 Fall allergies Health news Mental health Relax Sexual health Studies The Unwind Parenting Family health So mini ways Style and beauty It Figures Unapologetically Horoscopes Shopping Buying guides Food Travel Autos Gift ideas Buying guides Entertainment Celebrity TV Movies Music How to Watch Interviews Videos Finance My Portfolio News Latest Stock Market Originals The Morning Brief Economies Tariff Updates Premium News Earnings Tech Housing Crypto Markets Stocks: Most Actives Stocks: Gainers Stocks: Losers Trending Tickers Futures World Indices US Treasury Bonds Rates Currencies Crypto Top ETFs Top Mutual Funds Options: Highest Open Interest Options: Highest Implied Volatility Sectors Ba

Final Answer: 

Insights and alerts about significant market opportunities or threats for AAPL are crucial for making informed decisions. The Data Analyst has been delegated the task of continuously monitoring and analyzing market data for AAPL using statistical modeling and machine learning techniques. This will help in identifying trends and predicting market movements, ultimately leading to valuable insights for the team to act upon.

> Finished chain.
 [DEBUG]: [Crew Manager] Task output: Insights and alerts about significant market opportunities or threats for AAPL are crucial for making informed decisions. The Data Analyst has been delegated the task of continuously monitoring and analyzing market data for AAPL using statistical modeling and machine learning techniques. This will help in identifying trends and predicting market movements, ultimately leading to valuable insights for the team to act upon.
 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Develop and ref

Final Answer: 
Apple Inc. (AAPL) Stock Price, News, Quote & History - Yahoo Finance
Oops, something went wrong
News
Today's news
US Politics
World
Tech
Reviews and deals
Audio
Computing
Gaming
Health
Home
Phones
Science
TVs
Climate change
Health
Science Originals
The 360
Newsletters
Life
Health
COVID-19
Fall allergies
Health news
Mental health
Relax
Sexual health
Studies
The Unwind
Parenting
Family health
So mini ways
Style and beauty
It Figures
Unapologetically
Horoscopes
Shopping
Buying guides
Food
Travel
Autos
Gift ideas
Buying guides
Entertainment
Celebrity
TV
Movies
Music
How to Watch
Interviews
Videos
Finance
My Portfolio
News
Latest
Stock Market
Originals
The Morning Brief
Economies
Tariff Updates
Premium News
Earnings
Tech
Housing
Crypto
Markets
Stocks: Most Actives
Stocks: Gainers
Stocks: Losers
Trending Tickers
Futures
World Indices
US Treasury Bonds Rates
Currencies
Crypto
Top ETFs
Top Mutual Funds
Options: Highest Open Interest
Options: Highest Implied Volatility
Sectors
Ba

Final Answer: 
Based on the insights from the Data Analyst and the user-defined risk tolerance (Medium), as well as considering trading preferences (Day Trading), here are potential trading strategies for AAPL:

1. Short-term trading strategy: Utilize statistical modeling and machine learning techniques to identify short-term trends in AAPL stock price movements. Implement a strategy that involves frequent buying and selling of AAPL stocks based on these trends to capitalize on short-term price fluctuations.

2. Long-term investment strategy: Develop a long-term investment strategy for AAPL that aligns with the user's risk tolerance and preferences for Day Trading. This strategy should involve holding AAPL stocks for an extended period to benefit from potential long-term growth in the company.

3. Options trading strategy: Explore options trading as a strategy for AAPL, considering the user's risk tolerance and preferences. This strategy could involve buying call or put options on AAPL

I need to gather as much information as possible to provide a comprehensive risk analysis report for AAPL.

Action: Ask question to co-worker
Action Input: {"coworker": "Data Analyst", "question": "Can you provide detailed insights on the short-term trading strategy for AAPL?", "context": "I need this information to evaluate the risks associated with the proposed trading strategies and execution plans for AAPL."}

> Entering new CrewAgentExecutor chain...
I need to gather detailed insights on the short-term trading strategy for AAPL by analyzing market data in real-time.

Action: Read website content
Action Input: {"website_url": "https://www.marketwatch.com/investing/stock/aapl"} 

marketwatch.comPlease enable JS and disable any ad blocker

Action: Search the internet
Action Input: {"search_query": "short-term trading strategy for AAPL site:marketwatch.com"} 


Search results: Title: Another 'Volmageddon'? JPMorgan becomes the latest to ...
Link: https://www.marketwatch.com/story/anot

- Display the final result as Markdown.

In [16]:
from IPython.display import Markdown
Markdown(result)

Risk Analysis Report for AAPL:

After gathering insights from the Data Analyst on the short-term trading strategy for AAPL, it is evident that there are significant risks associated with executing these strategies. The potential risks include market volatility, liquidity issues, and regulatory changes that could impact the execution of trades for AAPL. 

To mitigate these risks, it is recommended to closely monitor market conditions, establish clear risk management protocols, and diversify the trading portfolio to reduce exposure to any single asset. Additionally, staying informed about any regulatory changes that could impact AAPL's trading activities is crucial to mitigate potential risks.

Overall, a cautious approach is advised when considering short-term trading strategies for AAPL, as the market conditions are dynamic and unpredictable. By carefully evaluating the risks and implementing appropriate mitigation strategies, the trading team can enhance the overall success of executing trades for AAPL.